# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from ddpg_agent import Agent
import torch
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=False)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size_env = brain.vector_action_space_size
print('Size of each action:', action_size_env)

# examine the state space 
states = env_info.vector_observations
state_size_env = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape, state_size_env))
print('The state for the first agent looks like:', states)

Number of agents: 2
Size of each action: 2
There are (2, 24) agents. Each observes a state with length: 24
The state for the first agent looks like: [[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.65278625 -1.5
  -0.          0.          6.83172083  6.         -0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.4669857  -1.5
   0.          0.         -6.83172083  6.          0.          0.        ]]


### 3. Train the Agent


In [ ]:
agent = Agent(state_size=state_size_env, action_size=action_size_env, num_agents=2, random_seed=0)
#agent = Agent(state_size=state_size_env, action_size=action_size_env, actor_chkpt_file="checkpoint_actor.pth", critic_chkpt_file="checkpoint_critic.pth", random_seed=0)
#agent_2 = Agent(state_size=state_size_env, action_size=action_size_env, random_seed=0)

def ddpg(n_episodes=25000, max_t=1000):
    scores_deque = deque(maxlen=100)
    scores_all = []
    balls_over = 0
    noise_scale = 1.0
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]     # reset the environment
        agent.reset()
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                         # initialize the score (for each agent)    
        actions = np.zeros((num_agents, action_size_env))
        #agent.reset()
        t = 0
        while True:
            
            actions = agent.act(states, True, noise_scale)

            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            done = env_info.local_done                         # see if episode finished
            scores += rewards                                  # update the score (for each agent
        
            if rewards[0] >= 0.1 or rewards[1] >= 0.1:
                balls_over += 1

            agent.step(states, actions, rewards, next_states, done)

            states = next_states
            
            if np.any(done):                                  # exit loop if episode finished
                break 
                
        noise_scale = noise_scale*0.99
        scores_avg = np.amax(scores)
        scores_deque.append(scores_avg)
        scores_all.append(scores_avg)
        #print(scores_agents)
        print('\rEpisode {}\tAverage Score: {:2.3f}\tScore: {:2.3f}\tBalls Over: {}        '.format(i_episode, np.mean(scores_deque), scores_avg, balls_over), end="")
        if i_episode % 200 == 0:
            print(agent.actor_scheduler.get_lr())
            print('\nEpisode {}\tAverage Score: {:2.3f}\tScore: {:2.3f}\tBalls Over: {}       '.format(i_episode, np.mean(scores_deque), scores_avg, balls_over), end="")
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor_{}.pth'.format(i_episode))
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic_{}.pth'.format(i_episode))   
    return scores_all

#scores = ddpg(n_episodes=10, max_t=130)


scores = ddpg()
print('scores len  {}'.format(len(scores)))

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()
fig.savefig('AverageScore.png')        

Episode 200	Average Score: 0.007	Score: 0.100	Balls Over: 11        [1.0101010101010103e-05]

Episode 400	Average Score: 0.031	Score: 0.000	Balls Over: 57        [1.0101010101010103e-05]

Episode 600	Average Score: 0.035	Score: 0.000	Balls Over: 145        [1.0101010101010103e-05]

Episode 800	Average Score: 0.027	Score: 0.000	Balls Over: 215        [1.0101010101010103e-05]

Episode 1000	Average Score: 0.052	Score: 0.100	Balls Over: 317        [1.0101010101010103e-05]

Episode 1200	Average Score: 0.071	Score: 0.100	Balls Over: 486        [1.0101010101010103e-05]

Episode 1400	Average Score: 0.083	Score: 0.100	Balls Over: 730        [1.0101010101010103e-05]

Episode 1600	Average Score: 0.162	Score: 0.200	Balls Over: 1153        [1.0101010101010103e-05]

Episode 1800	Average Score: 0.324	Score: 0.100	Balls Over: 2089        [1.0101010101010103e-05]

Episode 2000	Average Score: 0.524	Score: 0.200	Balls Over: 4310        [1.0101010101010103e-05]

Episode 2200	Average Score: 0.321	Score: 0.

Episode 16800	Average Score: 0.143	Score: 0.200	Balls Over: 101366        [1.0101010101010103e-05]

Episode 17000	Average Score: 0.121	Score: 0.100	Balls Over: 101760        [1.0101010101010103e-05]

Episode 17200	Average Score: 0.173	Score: 0.100	Balls Over: 102293        [1.0101010101010103e-05]

Episode 17400	Average Score: 0.777	Score: 0.100	Balls Over: 104329        [1.0101010101010103e-05]

Episode 17600	Average Score: 0.674	Score: 2.600	Balls Over: 107149        [1.0101010101010103e-05]

Episode 17800	Average Score: 0.759	Score: 0.300	Balls Over: 109614        [1.0101010101010103e-05]

Episode 18000	Average Score: 0.458	Score: 1.300	Balls Over: 111430        [1.0101010101010103e-05]

Episode 18200	Average Score: 0.328	Score: 0.000	Balls Over: 112848        [1.0101010101010103e-05]

Episode 18400	Average Score: 0.242	Score: 0.200	Balls Over: 113771        [1.0101010101010103e-05]

Episode 18600	Average Score: 0.247	Score: 0.200	Balls Over: 114685        [1.0101010101010103e-05]



When finished, you can close the environment.

In [ ]:
env.close()